In [1]:
import os
import sys
sys.path.append('../utils')
sys.path.append('../data')  

import glob

import numpy as np
import pandas as pd

from joblib import load, dump

from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder

from preprocessing import load_csv_data

In [2]:
# Lista de fișiere CSV de procesat
csv_files = glob.glob('../data/TrafficLabelling/*')
print(f"{len(csv_files)} fișiere găsite:")

8 fișiere găsite:


In [3]:
# Pregătește encoderul pentru labeluri
le_label = LabelEncoder()
model = SGDClassifier(random_state=42)

first_fit = True  # flag pentru primul CSV (unde setăm classes)

In [4]:
all_labels_set = set()

# Colectează toate etichetele din toate fișierele
for csv_file in csv_files:
    df = load_csv_data(csv_file)
    df.columns = df.columns.str.strip()
    df.rename(columns={'Label': 'label'}, inplace=True)
    all_labels_set.update(df['label'].unique())

# Inițializează encoderul și antrenează-l pe TOATE etichetele
le_label = LabelEncoder()
le_label.fit(list(all_labels_set))

first_fit = True  # dacă nu era definit

# Reîncepe bucla pentru procesare și antrenare
for csv_file in csv_files:
    print(f"\nProcesare fișier: {csv_file}")

    df = load_csv_data(csv_file)
    df.columns = df.columns.str.strip()
    df.rename(columns={'Label': 'label'}, inplace=True)

    X = df.drop(columns=['label'])
    y = df['label']

    X = X.replace([np.inf, -np.inf], np.nan).dropna()
    y = y.loc[X.index]

    if 'src_ip' in X.columns:
        X['src_ip'] = LabelEncoder().fit_transform(X['src_ip'])

    X = X.select_dtypes(include=['number'])
    y = le_label.transform(y)

    if first_fit:
        model.partial_fit(X, y, classes=le_label.transform(le_label.classes_))
        first_fit = False
    else:
        model.partial_fit(X, y)

    print(f"Antrenat pe {len(X)} instanțe.")


Procesare fișier: ../data/TrafficLabelling/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
Antrenat pe 288395 instanțe.

Procesare fișier: ../data/TrafficLabelling/Monday-WorkingHours.pcap_ISCX.csv
Antrenat pe 529481 instanțe.

Procesare fișier: ../data/TrafficLabelling/Friday-WorkingHours-Morning.pcap_ISCX.csv
Antrenat pe 190911 instanțe.

Procesare fișier: ../data/TrafficLabelling/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
Antrenat pe 286096 instanțe.

Procesare fișier: ../data/TrafficLabelling/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
Antrenat pe 225711 instanțe.

Procesare fișier: ../data/TrafficLabelling/Tuesday-WorkingHours.pcap_ISCX.csv
Antrenat pe 445645 instanțe.

Procesare fișier: ../data/TrafficLabelling/Wednesday-workingHours.pcap_ISCX.csv
Antrenat pe 691406 instanțe.

Procesare fișier: ../data/TrafficLabelling/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
Antrenat pe 170231 instanțe.


In [5]:
# Salvează modelul și encoderul
dump(model, '../models/model_sgd_cicids.joblib')
dump(le_label, '../models/label_encoder.joblib')
dump(model, '../models/model_sgd_cicids.pt')
print("\n Modelul incremental a fost complet antrenat și salvat.")


 Modelul incremental a fost complet antrenat și salvat.
